In [ ]:
from vastdb.session import Session
from dotenv import load_dotenv
import os
import pyspark.sql.functions as sf


load_dotenv()
vast_db_access = os.getenv("VAST_DB_ACCESS")
vast_db_secret = os.getenv("VAST_DB_SECRET")
endpoint = os.getenv("VAST_DB_ENDPOINT")
sess = Session(access=vast_db_access, secret=vast_db_secret, endpoint=endpoint)

In [ ]:
pos = spark.table("ndb.colabfit.dev.po_wip")
cos = spark.table("ndb.colabfit.dev.co_dataset_id_sets")
# css = spark.table('ndb.colabfit.dev.cs_wip')
dss = spark.table("ndb.colabfit.dev.ds_wip")

In [ ]:
import pyspark.sql.functions as sf

ocs = dss.select("id", "name").filter(sf.col("name").contains("OC20"))

In [ ]:
for id in [x["id"] for x in ocs.collect()]:
    with open("OC20_cos_dataset_ids2.txt", "a") as f:
        f.write(f"{id}\n\n")
    cos_ds_ids = cos.filter(sf.col("dataset_ids").contains(id)).select("dataset_ids")
    cos_ds_ids = cos_ds_ids.distinct().collect()
    with open("OC20_cos_dataset_ids2.txt", "a") as f:
        for x in cos_ds_ids:
            f.write(f'{x["dataset_ids"]}\n')

In [ ]:
# The following ids can be removed when we fix OC20 -- all instances are shared only between the existing OC20
# datasets, whether S2EF or IS2RES
# +-----------------+----------------------+
# |id               |name                  |
# +-----------------+----------------------+
# |DS_zdy2xz6y88nl_0|OC20_S2EF_train_200K  |
# |DS_dgop3abwq9ya_0|OC20_IS2RES_train     |
# |DS_7qi6dh0ig7sd_0|OC20_S2EF_train_2M    |
# |DS_otx1qc9f3pm4_0|OC20_S2EF_train_20M   |
# |DS_wmgdq06mzdys_0|OC20_S2EF_val_ood_cat |
# |DS_cgdsc3gxoamu_0|OC20_S2EF_val_ood_ads |
# |DS_wv9zv6egp9vk_0|OC20_S2EF_val_id      |
# |DS_889euoe7akyy_0|OC20_S2EF_val_ood_both|
# +-----------------+----------------------+
#  in addition, the id DS_rf10ovxd13ne_0 is one that should be removed as a failed ingest
#  of the 20 M split of S2EF from mongo.

In [ ]:
from pyspark.sql.types import ArrayType, StringType
from ast import literal_eval

unstring_and_make_set_list_udf = sf.udf(
    lambda x: str(list(set(literal_eval(x))), StringType())
)